In [ ]:
import pandapower as pp
import pandas as pd
import numpy as np
from pandapower.timeseries.output_writer import OutputWriter


# Install pandapower in a new environment by pip install pandapower==2.13.1
net = pp.from_pickle('./aggregated_grid_2018_with_generators_loads_costs_controllers.p')

out_path = "./AC_OPF/Results V2"
ow = OutputWriter(net, output_path=out_path, output_file_type=".csv", csv_separator=",")
# ow.remove_log_variable("res_gen", "vm_pu")

ext_grid_to_gen = False

In [ ]:
# TIMESTEP-WISE SIMULATION TO GET SIMULATED VALUES

# Simulate for a single timestep (t) using function run (pp.runpp, pp.runopp,...) and give the outputs at net.res_...
def get_simulated_values(t, run):
    for controller in net.controller.object:
        controller.time_step(net, t)

    run(net, verbose=True)

In [ ]:
# GET FROM SIMULATION

def get_from_simulation(res_csv, component_numbers):
    # Join simulation timeseries dataframes in a dataframe
    bus_timesteps = pd.concat([res_csv[f"{g}"] for g in component_numbers], axis=1)

    return bus_timesteps

In [ ]:
# GET FROM CONTROLLER

def get_from_controller(start_controller_number, component_numbers):
    # Join controller timeseries dataframes at the same bus in a dataframe
    bus_timesteps = pd.concat([net.controller.object[start_controller_number + g].data_source.to_dict()['df'].iloc[:,0] for g in component_numbers], axis=1)

    return bus_timesteps

In [ ]:
# GET AGGREGATED DATA

total_bus_numbers = net.bus.index

def get_aggregated_x(res_x, bus_gens, get_from):
    # res_x = csv for get_from_csv or start controller number for get_from_controller
    gen_busses_x = [] # List of timestep-wise gen x dataframes for each bus

    for bus in bus_gens:
        # Join timeseries dataframes (q_mvar, vm_pu and va_degree) at the same bus in a dataframe
        bus_timesteps = get_from(res_x, bus)

        # Add timestep-wise values together
        bus_timesteps_total = bus_timesteps.sum(axis=1)
        
        gen_busses_x.append(bus_timesteps_total)

    gen_x = pd.concat(gen_busses_x, axis=1) # Put busses together in one df (rows = timesteps, columns = buses)

    gen_x.columns = bus_gens.index # Column names to bus numbers

    gen_x = gen_x.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), axis=1, fill_value=0) # Fill missing bus numbers with q_mvar = 0
    
    return gen_x



# gen_busses_q_mvar = [] 
# gen_busses_vm_pu = []
# gen_busses_va_degree = []



# for bus in bus_gens:
#     # Join timeseries dataframes (q_mvar, vm_pu and va_degree) at the same bus in a dataframe
#     bus_timesteps = pd.concat([res_q_mvar[f"{g}"] for g in bus], axis=1)
#     bus_timesteps_vm = pd.concat([res_vm_pu[f"{vm}"] for vm in bus], axis=1)
#     bus_timesteps_va = pd.concat([res_va_degree[f"{va}"] for va in bus], axis=1)

#     # Add timestep-wise values together
#     bus_timesteps_total = bus_timesteps.sum(axis=1)
#     bus_timesteps_total_vm = bus_timesteps_vm.sum(axis=1)
#     bus_timesteps_total_va = bus_timesteps_va.sum(axis=1)

#     gen_busses_x.append(bus_timesteps_total)
#     gen_busses_vm_pu.append(bus_timesteps_total_vm)    
#     gen_busses_va_degree.append(bus_timesteps_total_va)

# gen_q_mvar = pd.concat(gen_busses_x, axis=1) # Put busses together in one df (rows = timesteps, columns = buses)
# gen_vm_pu = pd.concat(gen_busses_vm_pu, axis=1) # Put busses together in one df (rows = timesteps, columns = buses)
# gen_va_degree = pd.concat(gen_busses_va_degree, axis=1) # Put busses together in one df (rows = timesteps, columns = buses)

# gen_q_mvar.columns = bus_gens.index # Column names to bus numbers
# gen_vm_pu.columns = bus_gens.index # Column names to bus numbers
# gen_va_degree.columns = bus_gens.index # Column names to bus numbers

# gen_q_mvar = gen_q_mvar.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), axis=1, fill_value=0) # Fill missing bus numbers with q_mvar = 0
# gen_vm_pu = gen_vm_pu.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), axis=1, fill_value=0) # Fill missing bus numbers with q_mvar = 0
# gen_va_degree = gen_va_degree.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), axis=1, fill_value=0) # Fill missing bus numbers with q_mvar = 0

# gen_vm_pu.head(100)    


In [ ]:
# CREATE LOAD Q_MVAR

pf = 0.95 # Using a power factor 0.95
p_to_q = np.sin(np.arccos(pf))/pf # P to Q constant

loads_p_mw = get_from_controller(0, net.load.index) # All controller p_mw data

loads_q_mvar = loads_p_mw.mul(p_to_q)

# loads_q_mvar.head(50)


In [ ]:
# CREATE LOAD Q_MVAR CONTROLLER

for l in range(len(net.load.index)):
    controller_load_q_mvar = loads_p_mw.iloc[:, l].multiply(p_to_q)
    dataset = controller_load_q_mvar.to_frame()
    prof_name = dataset.columns[0]
    dataset = pp.timeseries.data_sources.frame_data.DFData(dataset)
    
    pp.control.controller.const_control.ConstControl(net, "load", "q_mvar", element_index=l, 
                                                 data_source=dataset, 
                                                 recycle={'trafo': False, 'gen': False, 'bus_pq': True}, 
                                                 profile_name=prof_name)

# net.controller

In [ ]:
# SET REASONABLE Q_MVAR BOUNDARIES FOR GEN

for i in net.gen.index:
    net.gen.loc[i, "min_q_mvar"] = -1000.0
    net.gen.loc[i, "max_q_mvar"] = 1000.0

In [ ]:
# SET REASONABLE Q_MVAR BOUNDARIES FOR EXT_GRID

for i in net.ext_grid.index:
    net.ext_grid.loc[i, "min_q_mvar"] = -5000.0
    net.ext_grid.loc[i, "max_q_mvar"] = 5000.0

net.ext_grid.loc[0, "type"] = "External Grid" # Fix ext_grid 0
net.ext_grid.loc[0, "p_mw"] = 0.0

# net.ext_grid

In [ ]:
# REPLACE SLACK BUSSES TO GENERATORS EXCEPT MAIN SLACK

dropped_ext_grid_busses = []
dropped_ext_grid_busses_active = []

for i, slack_bus in net.ext_grid.iterrows():
    if i != 7:
        pp.create_gen(net, bus=slack_bus["bus"], p_mw=slack_bus["p_mw"], vm_pu=1.0, name=slack_bus["name"], max_q_mvar=slack_bus["max_q_mvar"], 
                      min_q_mvar=slack_bus["min_q_mvar"], min_p_mw=slack_bus["min_p_mw"], max_p_mw=slack_bus["max_p_mw"], type=slack_bus["type"],
                      slack=False, controllable=True, in_service=slack_bus["in_service"])
        
        dropped_ext_grid_busses.append(slack_bus["bus"]) # Keeps track of which ext_grids are changed
        dropped_ext_grid_busses_active.append(slack_bus["in_service"]) # Keeps track of which ext_grids are changed        

net.ext_grid = net.ext_grid.drop([i for i in net.ext_grid.index if i != 7]) # Empty net.ext_grid
net.ext_grid = net.ext_grid.reset_index(drop=True) # Set right index for remaining main slack

ext_grid_to_gen = True # Set True if ext_grids are changed to gen, will be needed later

In [ ]:
# SET THEIR SLACK ON/OFF

for i in range(69, 78):
    net.gen.slack[i] = False
    net.gen.slack_weight[i] = 0.0

In [ ]:
# ADJUST COST FUNCTIONS

dropped_ext_grid_idx = [i for i in net.pwl_cost.index if i != 7]

for i in range(len(dropped_ext_grid_busses)):
    pp.create_pwl_cost(net, element=69 + i, et="gen", points=net.pwl_cost.points[dropped_ext_grid_idx[i]]) # Create gen type cost functions

net.pwl_cost.element[7] = 0
net.pwl_cost = net.pwl_cost.drop(dropped_ext_grid_idx) # Remove previous cost functions
net.pwl_cost = net.pwl_cost.reset_index(drop=True)

net.pwl_cost

In [ ]:
# LOG SIMULATION VARIABLES

ow.log_variable('res_bus', 'vm_pu') # Add logging for Bus
ow.log_variable('res_bus', 'va_degree') 
ow.log_variable('res_bus', 'p_mw') 
ow.log_variable('res_bus', 'q_mvar')

ow.log_variable('res_line', 'loading_percent') # Line
ow.log_variable('res_line', 'i_from_ka') 
ow.log_variable('res_line', 'i_to_ka') 
ow.log_variable('res_line', 'i_ka') 
ow.log_variable('res_line', 'p_from_mw')
ow.log_variable('res_line', 'q_from_mvar') 
ow.log_variable('res_line', 'p_to_mw') 
ow.log_variable('res_line', 'q_to_mvar') 
ow.log_variable('res_line', 'vm_from_pu') 
ow.log_variable('res_line', 'vm_to_pu') 
ow.log_variable('res_line', 'va_from_degree') 
ow.log_variable('res_line', 'va_to_degree') 

ow.log_variable('res_gen', 'p_mw') # Gen
ow.log_variable('res_gen', "q_mvar")
ow.log_variable('res_gen', "va_degree")
ow.log_variable('res_gen', "vm_pu")

ow.log_variable('res_sgen', 'p_mw') # Sgen
ow.log_variable('res_sgen', 'q_mvar')

ow.log_variable('res_load', 'p_mw') # Load
ow.log_variable('res_load', 'q_mvar')

ow.log_variable('res_ext_grid', 'p_mw') # Ext grid
ow.log_variable('res_ext_grid', 'q_mvar')

ow.log_variable('res_trafo', 'loading_percent') # Transformer
ow.log_variable('res_trafo', 'p_hv_mw')
ow.log_variable('res_trafo', 'p_hv_mvar')
ow.log_variable('res_trafo', 'p_lv_mw')
ow.log_variable('res_trafo', 'q_lv_mvar')
ow.log_variable('res_trafo', 'pl_mw')
ow.log_variable('res_trafo', 'ql_mvar')
ow.log_variable('res_trafo', 'i_hv_ka')
ow.log_variable('res_trafo', 'i_lv_ka')
ow.log_variable('res_trafo', 'vm_hv_pu')
ow.log_variable('res_trafo', 'va_hv_degree')
ow.log_variable('res_trafo', 'vm_lv_pu')
ow.log_variable('res_trafo', 'va_lv_degree')

In [ ]:
# GET TIMESERIES SIMULATION RESULTS

# pp.timeseries.run_timeseries(net, run=pp.runopp, continue_on_divergence=True) # Ignore divergence 

starting_step = 6912
total_time = 72
pp.timeseries.run_timeseries(net, run=pp.runopp, continue_on_divergence=True, time_steps=range(starting_step, starting_step + total_time))

In [ ]:
# *OR* GET INDIVIDUAL SIMULATION STEP

get_simulated_values(t=6912, run=pp.runopp)

In [ ]:
# SWAPPING SLACK BUS (35)

def swap_slack(df, slack_number): # Switch slack bus position

    copy = df.iloc[:, 0].copy()
    df.iloc[:, 0] = df.iloc[:, slack_number]
    df.iloc[:, slack_number] = copy
    
slack_bus = 35

In [ ]:
# GET GENERATION P_MW

if ext_grid_to_gen:
    gen_active = net.gen.drop(net.gen.tail(len(dropped_ext_grid_busses)).index) # Empty net.ext_grid
else:
    gen_active = net.gen

gen_active = gen_active.loc[net.gen["in_service"] == True] # Drop unused generators
bus_gens = gen_active.groupby("bus").apply(lambda x: x.index.tolist()) # Group generators by bus

res_gen_p_mw = pd.read_csv(out_path + "/res_gen/p_mw.csv") # Read from simulation result

gen_p_mw = get_aggregated_x(res_gen_p_mw, bus_gens, get_from_simulation) # Get bus-aggregated generation

swap_slack(gen_p_mw, slack_bus)

gen_p_mw.head(10)

In [ ]:
# GET BUS VM_PU

res_vm_pu = pd.read_csv(out_path + "/res_bus/vm_pu.csv")

vm_pu = res_vm_pu.drop(columns=["Unnamed: 0"])

swap_slack(vm_pu, slack_bus)

vm_pu.head(20)

In [ ]:
# GET LOAD P_MW

bus_loads = net.load.groupby("bus").apply(lambda x: x.index.tolist()) # Group loads by bus

load_p_mw = get_aggregated_x(0, bus_loads, get_from_controller)

if ext_grid_to_gen:
    load_p_mw = load_p_mw.iloc[starting_step: starting_step + total_time].reset_index(drop=True)
    # Fix if intended to use for no ext_grid_to_gen less than one year

# # Print load P_MW for forecasting:
# load_p_mw.to_csv(out_path + "/total_load.csv")

swap_slack(load_p_mw, slack_bus)

load_p_mw.head(10)

In [ ]:
# GET LOAD Q_MVAR

load_q_mvar = load_p_mw.mul(p_to_q)

# load_q_mvar.head(50)

In [ ]:
# GET SGEN P_MW

sgen_active = net.sgen.loc[net.sgen["in_service"] == True] # Drop unused sgens
bus_sgens = sgen_active.groupby("bus").apply(lambda x: x.index.tolist()) # Group sgens by bus

res_sgen_p_mw = pd.read_csv(out_path + "/res_sgen/p_mw.csv") # Read from simulation result

sgen_p_mw = get_aggregated_x(res_sgen_p_mw, bus_sgens, get_from_simulation) # Get bus aggregated sgen

swap_slack(sgen_p_mw, slack_bus)

sgen_p_mw.head(50)

In [ ]:
# GET SGEN Q_MVAR

res_sgen_q_mvar = pd.read_csv(out_path + "/res_sgen/q_mvar.csv") # Read from simulation result

sgen_q_mvar = get_aggregated_x(res_sgen_q_mvar, bus_sgens, get_from_simulation) # Get bus aggregated sgen

swap_slack(sgen_q_mvar, slack_bus)

sgen_q_mvar.head(50)

In [ ]:
# GET BUS ANGLE

res_va_degree = pd.read_csv(out_path + "/res_bus/va_degree.csv")
va_degree = res_va_degree.drop(columns=["Unnamed: 0"])

swap_slack(va_degree, slack_bus)

va_degree.head(20)

In [ ]:
# GET SLACK WEIGHT (AND SLACK)

slack_active = net.ext_grid.loc[net.ext_grid["in_service"] == True]

slack_weight = [1 if i in slack_active.bus.to_list() else 0 for i in total_bus_numbers]

if ext_grid_to_gen:
    for i, b in enumerate(dropped_ext_grid_busses):
        is_active = dropped_ext_grid_busses_active[i]
        slack_weight[b] = 1 if is_active else 0
    
slack_weight[0], slack_weight[slack_bus] = slack_weight[slack_bus], slack_weight[0]

slack = slack_weight # Slack yes or no

print(slack_weight)

In [ ]:
# GET EMPTY BUSES

active_bus = set(net.load.bus.to_list())
active_bus.update(gen_active.bus.to_list())
active_bus.update(sgen_active.bus.to_list())
active_bus.update(slack_active.bus.to_list())

if ext_grid_to_gen:
    active_ext_grid_busses = [dropped_ext_grid_busses[i] for i in range(len(dropped_ext_grid_busses)) if dropped_ext_grid_busses_active[i]]
    active_bus.update(active_ext_grid_busses)

empty_bus = [bus for bus in total_bus_numbers if bus not in active_bus]

for i in range(len(empty_bus)):
    if empty_bus[i] == 0:
        empty_bus[i] = slack_bus
    elif empty_bus[i] == slack_bus:
        empty_bus[i] = 0

print(empty_bus)

In [ ]:
# GET BUS TYPE

# 2 if PV (or slack), 3 if PQ (or empty/not in service --> P=0, Q=0)
bus_type = pd.Series([2 if (i in bus_gens.index or i in slack_active.bus.to_list()) 
                   else 3 for i in total_bus_numbers]) 

if ext_grid_to_gen:
    for b in active_ext_grid_busses:
        bus_type[b] = 2 # Slack node

bus_type[0], bus_type[slack_bus] = 1, bus_type[0] # Main slack is 1

print(bus_type)

In [ ]:
# JOIN LOAD AND SGEN

load_sgen_p_mw = load_p_mw.subtract(sgen_p_mw)
load_sgen_q_mvar = load_q_mvar.subtract(sgen_q_mvar)

load_sgen_p_mw.head(5)

In [ ]:
# GET MAX AND MIN GEN MVAR

min_gen_q_mvar = gen_active.groupby("bus")["min_q_mvar"].sum()
min_gen_q_mvar = min_gen_q_mvar.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), fill_value=None) # Fill missing buses with none

max_gen_q_mvar = gen_active.groupby("bus")["max_q_mvar"].sum()
max_gen_q_mvar = max_gen_q_mvar.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), fill_value=None) # Fill missing buses with none

min_gen_q_mvar

In [ ]:
# GET USER SOLAR AND WIND BUSSES

extra_bus = pd.Series([2, 1.0, None, None, None, 0, 0, 1000.0, -1000.0, 0, 0])
extra_buses = pd.DataFrame([extra_bus, extra_bus])
extra_buses.columns = ["Type", "PU Volt", "Angle (Deg)", "Gen MW", "Gen Mvar", "Load MW", "Load Mvar", 
                       "Gen Mvar(max)", "Gen Mvar(min)", "Slack yes,no", "Slack Weight"]
extra_buses

In [ ]:
# TRANSFORM TO P.U.

def to_pu(power, sbase=100.0):
    return np.divide(power, sbase)

In [ ]:
# GETTING ALL DATA FOR ONE TIMESTEP

nones = [None for _ in total_bus_numbers]

def get_timestep(t):
    timestep = pd.DataFrame(data={"Type": bus_type, "PU Volt": vm_pu.iloc[t].values, "Angle (Deg)": va_degree.iloc[t].values, 
    "Gen MW": to_pu(gen_p_mw.iloc[t].values), "Gen Mvar": nones, "Load MW": to_pu(load_sgen_p_mw.iloc[t].values), 
    "Load Mvar": to_pu(load_sgen_q_mvar.iloc[t].values), "Gen Mvar(max)": to_pu(max_gen_q_mvar), 
    "Gen Mvar(min)": to_pu(min_gen_q_mvar), "Slack yes,no": slack, "Slack Weight": slack_weight})
    
    timestep = pd.concat([timestep, extra_buses], axis=0, ignore_index=True) # Add solar and wind bus
    
    return timestep

get_timestep(0).head(50)


In [ ]:
# CREATING THE CSV WITH ALL TIMESTEPS

def create_csv(t_init, t_duration, index=True):
    result = pd.concat([get_timestep(t) for t in range(t_init, t_init + t_duration)], axis=0)
    
    result.to_csv(out_path + "/busInputData.csv", index=index)

In [ ]:
# GET NON-CONVERGED TIMESTEPS INDEXES

non_conv_mask = (vm_pu == 0).all(axis=1)
non_conv_idx = vm_pu.index[non_conv_mask].to_list() # Non converged timesteps index

conv_periods = [non_conv_idx[i+1] - non_conv_idx[i] for i in range(len(non_conv_idx) - 1)] # Periods of convergence
from_to_conv = [[non_conv_idx[i], non_conv_idx[i+1]] for i in range(len(non_conv_idx) - 1)]
convergences = dict(zip(conv_periods, from_to_conv))

conv_periods_sorted = conv_periods
conv_periods_sorted.sort(reverse=True)
conv = {i: convergences[i] for i in conv_periods_sorted} # {Length period of convergence: [start, end]}

print(conv)

# Result is {186: [7982, 8168], 149: [4829, 4978], 141: [8364, 8505], 131: [6770, 6901], 121: [5959, 6080], 113: [7391, 7504], 112: [7746, 7858], 110: [5775, 5885], 106: [728, 834], 105: [6080, 6185], 104: [6911, 7015], 103: [136, 239], 100: [6406, 6506], 98: [3519, 3617], 96: [1329, 1425], 93: [4173, 4266], 91: [8271, 8362], 89: [1885, 1974], 84: [1069, 1153], 82: [7266, 7348], 80: [3711, 3791], 79: [3440, 3519], 77: [6563, 6640], 76: [8511, 8587], 75: [7015, 7090], 74: [7136, 7210], 72: [5141, 5213], 70: [426, 496], 68: [7911, 7979], 63: [5078, 5141], 62: [7576, 7638], 61: [2205, 2266], 60: [8587, 8647], 59: [3835, 3894], 56: [7210, 7266], 55: [1561, 1616], 54: [9, 63], 53: [7638, 7691], 52: [6326, 6378], 51: [8220, 8271], 50: [4743, 4793], 48: [7691, 7739], 47: [2933, 2980], 46: [919, 965], 45: [3234, 3279], 44: [7504, 7548], 43: [7348, 7391], 42: [2987, 3029], 41: [5653, 5694], 40: [4357, 4397], 38: [4077, 4115], 37: [2474, 2511], 36: [4041, 4077], 34: [7877, 7911], 33: [6506, 6539], 32: [6185, 6217], 31: [3898, 3929], 29: [5460, 5489], 28: [7548, 7576], 27: [6679, 6706], 26: [5312, 5338], 25: [5694, 5719], 24: [6640, 6664], 23: [7113, 7136], 22: [5593, 5615], 20: [4637, 4657], 19: [613, 632], 18: [8174, 8192], 17: [5885, 5902], 16: [8647, 8663], 15: [8205, 8220], 14: [5271, 5285], 13: [8192, 8205], 12: [5489, 5501], 11: [6759, 6770], 10: [6901, 6911], 9: [5758, 5767], 8: [6398, 6406], 7: [6296, 6303], 6: [8168, 8174], 5: [8506, 8511], 4: [5348, 5352], 3: [5432, 5435], 2: [8362, 8364], 1: [8505, 8506]}

In [ ]:
create_csv(6912, 48, index=False) # 6912/24 = 288th day 1st hour

In [ ]:
# TESTING OPF

res_load_p_mw = pd.read_csv(out_path + "/res_load/p_mw.csv") # Read from simulation result
res_load_q_mvar = res_load_p_mw.mul(p_to_q)

res_gen_vm_pu = pd.read_csv(out_path + "/res_gen/vm_pu.csv")

res_ext_grid_vpu = res_vm_pu.iloc[:, 36:] # Wrong
res_ext_grid_vm_pu = pd.concat([res_ext_grid_vpu.iloc[:, 3:], res_ext_grid_vpu.iloc[:, :3]], axis=1)

res_ext_grid_vadeg = res_va_degree.iloc[:, 36:]
res_ext_grid_va_degree = pd.concat([res_ext_grid_vadeg.iloc[:, 3:], res_ext_grid_vadeg.iloc[:, :3]], axis=1)
# res_ext_grid_vm_pu.head()


def run_simulation(t_init=0, t_final=8760, function=pp.runpp):

    # init_load = net.load[["p_mw", "q_mvar"]]

    net.ext_grid["vm_pu"] = np.ones(len(net.ext_grid))
    net.ext_grid["va_degree"] = np.zeros(len(net.ext_grid))

    res_lp = []
    res_lq = []

    res_gp = []
    res_gq = []
    res_gvm = []
    res_gva = []

    res_sgp = []
    res_sgq = []

    res_bp = []
    res_bq = []
    res_bvm = []
    res_bva = []

    res_l = []

    for t in range(t_init, t_final):
        net.load["p_mw"] = res_load_p_mw.iloc[t, 1:].values
        net.load["q_mvar"] = res_load_q_mvar.iloc[t, 1:].values

        net.gen["p_mw"] = res_gen_p_mw.iloc[t, 1:].values
        net.gen["vm_pu"] = res_gen_vm_pu.iloc[t, 1:].values

        net.sgen["p_mw"] = res_sgen_p_mw.iloc[t, 1:].values
        net.sgen["q_mvar"] = res_sgen_q_mvar.iloc[t, 1:].values

        # print(res_ext_grid_vm_pu.iloc[t].values)

        net.ext_grid["vm_pu"] = res_ext_grid_vm_pu.iloc[t].values
        net.ext_grid["va_degree"] = res_ext_grid_va_degree.iloc[t].values

        function(net)

        res_lp.append(net.res_load["p_mw"].copy().to_frame().T) # load_p_mw
        res_lq.append(net.res_load["q_mvar"].copy().to_frame().T) # load_q_mvar

        res_gp.append(net.res_gen["p_mw"].copy().to_frame().T) # gen_p_mw
        res_gq.append(net.res_gen["q_mvar"].copy().to_frame().T) # gen_q_mvar
        res_gvm.append(net.res_gen["vm_pu"].copy().to_frame().T) # gen_vm_pu
        res_gva.append(net.res_gen["va_degree"].copy().to_frame().T) # gen_q_mvar

        res_sgp.append(net.res_sgen["p_mw"].copy().to_frame().T) # gen_p_mw
        res_sgq.append(net.res_sgen["q_mvar"].copy().to_frame().T) # gen_p_mw

        res_bp.append(net.res_bus["p_mw"].copy().to_frame().T) # gen_q_mvar
        res_bq.append(net.res_bus["q_mvar"].copy().to_frame().T) # gen_q_mvar
        res_bvm.append(net.res_bus["vm_pu"].copy().to_frame().T) # gen_q_mvar
        res_bva.append(net.res_bus["va_degree"].copy().to_frame().T) # gen_q_mvar

        res_l.append(net.res_line["loading_percent"].copy().to_frame().T) # gen_q_mvar

    
    res_lp = pd.concat(res_lp, axis=0)
    res_lq = pd.concat(res_lq, axis=0)

    res_gp = pd.concat(res_gp, axis=0)
    res_gq = pd.concat(res_gq, axis=0)
    res_gvm = pd.concat(res_gvm, axis=0)
    res_gva = pd.concat(res_gva, axis=0)

    res_sgp = pd.concat(res_sgp, axis=0)
    res_sgq = pd.concat(res_sgq, axis=0)

    res_bp = pd.concat(res_bp, axis=0)
    res_bq = pd.concat(res_bq, axis=0)
    res_bvm = pd.concat(res_bvm, axis=0)
    res_bva = pd.concat(res_bva, axis=0)

    res_l = pd.concat(res_l, axis=0)

    return {"load": [res_lp, res_lq], "gen": [res_gp, res_gq, res_gvm, res_gva], "sgen": [res_sgp, res_sgq], 
            "bus": [res_bp, res_bq, res_bvm, res_bva], "line": [res_l]}


results = run_simulation(t_init=6912, t_final=6912+48, function=pp.runpp)


In [ ]:
results["line"][0].head()